In [13]:
import random
import networkx as nx
import itertools
import time

In [14]:
def is_resolving_set(B, G):
    distances = {}
    for v in G.nodes:
        distances[v] = [nx.shortest_path_length(G, v, u) for u in B]
    
    for v1, v2 in itertools.combinations(G.nodes, 2):
        if distances[v1] == distances[v2]:
            return False
    return True

In [15]:
def load_graph(file_path):
    G = nx.Graph() 

    with open(file_path, "r") as file:
        for line in file:
            parts = line.split()
            
            node1 = int(parts[0]) 
            node2 = int(parts[1])  
            G.add_edge(node1, node2) 
    return G

In [16]:
def load_gcol(file_path):
    G = nx.Graph() 

    with open(file_path, "r") as file:
        for line in file:
            parts = line.split()
            if parts[0] == 'e':
                node1 = int(parts[1]) 
                node2 = int(parts[2])  
                G.add_edge(node1, node2) 
    return G

In [17]:
def generate_population(size,G):
    return [[random.randint(0, 1) for _ in range(len(G.nodes))] for _ in range(size)]

In [18]:
def fitness(individual, G, shortest_paths):
    resolving_set = {i for i, bit in enumerate(individual) if bit == 1}

    if not resolving_set:
        return float("inf")

    unresolved_pairs = 0
    for u, v in itertools.combinations(G.nodes, 2):
        if not any(abs(shortest_paths[u][j] - shortest_paths[v][j]) > 0 for j in resolving_set if j in shortest_paths[u]):
            unresolved_pairs += 1  # Par (u, v) nije razrešen

    return len(resolving_set) + 100 * unresolved_pairs  # Penalizacija nerešenih parova

In [19]:
def tournament_selection(population, G, k=3):
    selected = random.sample(population, k)
    return min(selected, key=lambda ind: fitness(ind, G))

In [20]:
def weighted_selection(population, G,shortest_paths):
    fitness_values = [1 / (1 + fitness(ind, G,shortest_paths)) for ind in population]  # Inverzija jer tražimo minimu
    return random.choices(population, weights=fitness_values, k=2)  # Bira 2 roditelja

In [21]:
def crossover(parent1, parent2,G ):
    point = random.randint(1, len(G.nodes) - 1)
    return parent1[:point] + parent2[point:], parent2[:point] + parent1[point:]

In [22]:
def mutate(individual, mutation_rate=0.05):
    new_individual = individual[:]
    for i in range(len(new_individual)):
        if new_individual[i] == 1 and random.random() < mutation_rate:
            new_individual[i] = 0 
    return new_individual

In [23]:
def genetic_algorithm(G, shortest_paths,stop,generations, pop_size=30, crossover_rate=0.5, mutation_rate=0.1):
    population = generate_population(pop_size, G)

    # Inicijalizuj najbolju jedinku do sada
    global_best_solution = None
    global_best_fitness = float('inf')

    for gen in range(generations):
        new_population = []

        for _ in range(pop_size // 2):
            parent1, parent2 = weighted_selection(population, G, shortest_paths)

            # Ukrštanje
            if random.random() < crossover_rate:
                child1, child2 = crossover(parent1, parent2, G)
            else:
                child1, child2 = parent1[:], parent2[:]  # koristi kopije

            # Mutacija
            child1 = mutate(child1, mutation_rate)
            child2 = mutate(child2, mutation_rate)

            new_population.extend([child1, child2])

        population = new_population

        # Pronalazak najbolje jedinke u ovoj generaciji
        best_solution = min(population, key=lambda ind: fitness(ind, G, shortest_paths))
        best_fitness = fitness(best_solution, G, shortest_paths)

        print(f"Generacija {gen + 1}, Najbolji fitnes: {best_fitness}")

        # Ažuriranje globalno najboljeg rešenja
        if best_fitness < global_best_fitness:
            global_best_fitness = best_fitness
            global_best_solution = best_solution[:]
            no_improvement = 0  # Resetuj brojač
        else:
            no_improvement += 1

        if no_improvement >= stop:
            print(f"Prekid: nema poboljšanja tokom {stop} generacija.")
            break

    optimal_set = {i for i, bit in enumerate(global_best_solution) if bit == 1}
    print("\nOptimalni rešavajući skup:", optimal_set, len(optimal_set))
    return optimal_set

In [24]:
G = load_graph('grafovi\srednji grafovi\keller4.txt')
if not nx.is_connected(G):
    print("Graf nije povezan!")
    largest_cc = max(nx.connected_components(G), key=len)
    G = G.subgraph(largest_cc).copy()

start_time1 = time.time()
shortest_paths = dict(nx.all_pairs_shortest_path_length(G))
rez = genetic_algorithm(G,shortest_paths,3,500)
end_time1 = time.time()
print("Vrijeme ", end_time1-start_time1)
print(is_resolving_set(rez,G))

<>:1: SyntaxWarning: invalid escape sequence '\s'
<>:1: SyntaxWarning: invalid escape sequence '\s'
C:\Users\Korisnik\AppData\Local\Temp\ipykernel_387328\3679822134.py:1: SyntaxWarning: invalid escape sequence '\s'
  G = load_graph('grafovi\srednji grafovi\keller4.txt')


Generacija 1, Najbolji fitnes: 64
Generacija 2, Najbolji fitnes: 56
Generacija 3, Najbolji fitnes: 51
Generacija 4, Najbolji fitnes: 46
Generacija 5, Najbolji fitnes: 42
Generacija 6, Najbolji fitnes: 34
Generacija 7, Najbolji fitnes: 28
Generacija 8, Najbolji fitnes: 24
Generacija 9, Najbolji fitnes: 21
Generacija 10, Najbolji fitnes: 21
Generacija 11, Najbolji fitnes: 20
Generacija 12, Najbolji fitnes: 19
Generacija 13, Najbolji fitnes: 18
Generacija 14, Najbolji fitnes: 18
Generacija 15, Najbolji fitnes: 17
Generacija 16, Najbolji fitnes: 16
Generacija 17, Najbolji fitnes: 17
Generacija 18, Najbolji fitnes: 17
Generacija 19, Najbolji fitnes: 17
Prekid: nema poboljšanja tokom 3 generacija.

Optimalni rešavajući skup: {34, 162, 5, 40, 105, 42, 141, 144, 20, 55, 93, 23, 122, 61, 94, 31} 16
Vrijeme  52.6687958240509
True
